## Load necessary libraries

In [1]:
import pickle

model = pickle.load(open("/var/tmp/xhorvat9/ltr_bert/NewClassifiers/Superfamily/BERT/LTRBERT_family_classifier.b", "rb"))

2023-10-16 09:48:30.697410: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 09:48:31.239469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
import pickle 
embeddings, labels, seq_ids = pickle.load(open("/var/tmp/xhorvat9/ltr_bert/NewClassifiers/Superfamily/BERT/LTRBERT_superfamily_classifier_embeddings.b_average", "rb"))

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, random_state=42, test_size=0.1)

## Train the model on BERT embeddings

In [9]:
import tensorflow as tf
import keras
import numpy as np
nn = keras.models.Sequential()
nn.add(keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(768,1)))
nn.add(keras.layers.MaxPooling1D(pool_size=2))
nn.add(keras.layers.Flatten())
nn.add(keras.layers.Dense(units=128, activation='relu'))
nn.add(keras.layers.Dropout(0.5)) 
nn.add(keras.layers.Dense(units=1, activation='sigmoid'))

nn.compile(loss='binary_crossentropy', optimizer=keras.optimizers.AdamW(), metrics=['binary_accuracy'], weighted_metrics=['binary_accuracy'])

nn.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_test, np.array(y_test)), callbacks=[keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/10


2804/2804 [==============================] - 53s 19ms/step - loss: 0.3414 - binary_accuracy: 0.8598 - weighted_binary_accuracy: 0.8598 - val_loss: 0.3273 - val_binary_accuracy: 0.8593 - val_weighted_binary_accuracy: 0.8593
Epoch 2/10
2804/2804 [==============================] - 53s 19ms/step - loss: 0.3270 - binary_accuracy: 0.8625 - weighted_binary_accuracy: 0.8625 - val_loss: 0.3248 - val_binary_accuracy: 0.8608 - val_weighted_binary_accuracy: 0.8608
Epoch 3/10
2804/2804 [==============================] - 54s 19ms/step - loss: 0.3222 - binary_accuracy: 0.8641 - weighted_binary_accuracy: 0.8641 - val_loss: 0.3227 - val_binary_accuracy: 0.8650 - val_weighted_binary_accuracy: 0.8650
Epoch 4/10
2804/2804 [==============================] - 54s 19ms/step - loss: 0.3184 - binary_accuracy: 0.8659 - weighted_binary_accuracy: 0.8659 - val_loss: 0.3131 - val_binary_accuracy: 0.8614 - val_weighted_binary_accuracy: 0.8614
Epoch 5/10
2804/2804 [==============================] - 55s 19ms/step - los

In [10]:
nn.save('CNN_BERT_pool_model')

INFO:tensorflow:Assets written to: CNN_BERT_pool_model/assets


INFO:tensorflow:Assets written to: CNN_BERT_pool_model/assets


## Test the model

In [6]:
test_embeddings, test_labels = pickle.load(open("/var/tmp/xhorvat9/ltr_bert/NewClassifiers/Superfamily/BERT/LTRBERT_superfamily_classifier_embeddings_TEST.b_average", "rb"))
predictions = nn.predict(test_embeddings)

393/393 [==============================] - 0s 887us/step


In [10]:
pred = [1 if x > 0.5 else 0 for x in predictions]

In [22]:
import pandas as pd
import Bio.SeqIO as SeqIO
max_len=512
seq_ids = [rec.id for rec in SeqIO.parse("/var/tmp/xhorvat9/ltr_bert/FASTA_files/test_LTRs.fasta", "fasta") if len(rec.seq) >= max_len and len(rec.seq) > 0 and rec.description.split()[3] != "NAN"]
pd.DataFrame({"ID": seq_ids, "actual": test_labels, "pred": pred}).to_csv("test_predictions.csv", index=False)

In [18]:
import numpy as np
(np.array(test_labels) == pred).astype(int).sum()/len(test_labels)

0.8787396562698918

In [47]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('zhihan1996/DNA_bert_6')
model = BertForSequenceClassification.from_pretrained('xhorvat9/LTRBERT_superfamily_classifier_512', num_labels=2)

if torch.cuda.is_available():    

    device = torch.device("cuda")

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw